In [81]:
import os
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
from src.data_loader.data_set import Data_Set
from src.data_loader.utils import get_train_val_split
from src.constants import MASTER_THESIS_DIR, FREIHAND_DATA
from src.utils import read_json
import matplotlib.pyplot as plt
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display
import copy
from easydict import EasyDict as edict
from src.utils import read_json
from src.visualization.visualize import plot_hand
import cv2
from src.data_loader.utils import convert_2_5D_to_3D, convert_to_2_5D

In [111]:
train_param = edict(
    read_json(f"{MASTER_THESIS_DIR}/src/experiments/config/training_config.json")
)
train_param = edict(
    read_json(f"{MASTER_THESIS_DIR}/src/experiments/config/training_config.json")
)
train_param.augmentation_order = []  # ['rotate','crop', 'gaussian_blur','resize']
train_param.augmentation_params.crop_margin= 1.5
# train_param.augmentation_params.hue_factor_range=[0.1,1]
# train_param.augmentation_params.sat_factor_range=[0.1,1]
# train_param.augmentation_params.value_factor_beta_range= [0,0]
# train_param.augmentation_params.value_factor_alpha_range= [0.1 ,.1]
train_param.augmentation_flags = {
    "color_drop": False,
    "color_jitter": False,
    "crop": False,
    "cut_out": False,
    "flip": False,
    "gaussian_blur": False,
    "random_crop": False,
    "resize": False,
    "rotate": False,
}

train_data = Data_Set(
    config=train_param,
    transform=None,
    train_set=True,
    experiment_type="supervised",
)
val_data = copy.copy(train_data)
val_data.is_training(False)

train_data_loader, val_data_loader = get_train_val_split(
    train_data,
    batch_size=train_param.batch_size,
    num_workers=train_param.num_workers,
)

In [112]:
@interact(
    idx=widgets.IntSlider(min=1, max=3000, step=5, value=3))
def visualize(idx):
    train_data.experiment_type = "supervised"
    sample = train_data[idx]
    joints = sample["joints"]
    img = sample["image"]
#     print(img.numpy())
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)
    plt.imshow(transforms.ToPILImage()(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)))
    plt.xticks([])
    plt.yticks([])
#     plt.ylim([120 ,0])
#     plt.xlim([-10, 110])
#     plot_hand(ax, joints,linewidth='8',)
#     plt.grid(True)
#     plt.savefig("hand2.png",bbox_inches='tight')

interactive(children=(IntSlider(value=3, description='idx', max=3000, min=1, step=5), Output()), _dom_classes=…

In [80]:
@interact(
    idx=widgets.IntSlider(min=1, max=3000, step=5, value=476))
def visualize(idx):
    train_data.experiment_type = "supervised"
    train_data.augmenter.rotate=True
    train_data.augmenter.color_jitter=True
    sample = train_data[idx]
    joints = sample["joints"]
    img = sample["image"]
#     print(img.numpy())
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)
    plt.imshow(transforms.ToPILImage()(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)))
    plt.xticks([])
    plt.yticks([])
#     plt.ylim([120 ,0])
#     plt.xlim([-10, 110])
#     plot_hand(ax, joints,linewidth='8',)
#     plt.grid(True)
    plt.savefig("hand_augmented.png",bbox_inches='tight')

interactive(children=(IntSlider(value=476, description='idx', max=3000, min=1, step=5), Output()), _dom_classe…

In [103]:
import numpy as np
import math as m
  
def Rx(theta):
  return np.matrix([[ 1, 0           , 0           ],
                   [ 0, m.cos(theta),-m.sin(theta)],
                   [ 0, m.sin(theta), m.cos(theta)]])
def Ry(theta):
  return np.matrix([[ m.cos(theta), 0, m.sin(theta)],
                   [ 0           , 1, 0           ],
                   [-m.sin(theta), 0, m.cos(theta)]])
def Rz(theta):
  return np.matrix([[ m.cos(theta), -m.sin(theta), 0 ],
                   [ m.sin(theta), m.cos(theta) , 0 ],
                   [ 0           , 0            , 1 ]])

matrix([[ 1.        ,  0.        ,  0.        ],
        [ 0.        ,  0.15425145,  0.98803162],
        [ 0.        , -0.98803162,  0.15425145]])

In [113]:
idx = 2121
train_data.experiment_type = "supervised"
train_data.augmenter.rotate=False
train_data.augmenter.color_jitter=False
train_data.augmenter.resize=True
sample = train_data[idx]

In [115]:
@interact(
#     idx=widgets.IntSlider(min=1, max=3000, step=5, value=1),
angle=widgets.IntSlider(min=1, max=360, step=5, value=1))
def visualize(angle):
    joints , scale= convert_to_2_5D(sample["K"],sample["joints3D"] @ torch.Tensor(Rz(angle)) )
#     print(joints)
    img = sample["image"]
#     print(img.numpy())
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)
#     plt.imshow(transforms.ToPILImage()(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)))
    plt.xticks([])
    plt.yticks([])
#     plt.ylim([120 ,0])
#     plt.xlim([-10, 110])
    plot_hand(ax, joints,linewidth='8',)
#     plt.grid(True)

interactive(children=(IntSlider(value=1, description='angle', max=360, min=1, step=5), Output()), _dom_classes…